In [7]:
# Import dependencies
import pandas as pd
import matplotlib as plt
import numpy as np
import requests
import time
import scipy.stats import linregress


# Import keys
from config import w_api_key
from config import gkey


In [14]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "Minneapolis"


# Build query URL
query_url = url + "appid=" + w_api_key + "&q=" + city
# print(query_url)
# Hooray!  I'm talking to the data source and my call is working.

In [21]:
# So, I'm pretty sure I need to use the citypy script 
# to randomly select 500 cities
# So how do I do that...




In [22]:
# From Stephanie

# Use citipy to find cities near coordinates

#Import citipy
from citipy import citipy

cities = []
countries = []

for lat_long in lat_long:
    city = citipy.nearest_city(lat_long[0], lat_long[1]).city_name
    country = citipy.nearest_city(lat_long[0], lat_long[1]).country_code
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
print(len(cities))
print(len(countries))

ModuleNotFoundError: No module named 'citipy'